In [ ]:
!pip install \
    pandas \
    xarray \
    matplotlib \
    cftime \
    zarr \
    fsspec \
    cartopy \
    s3fs \
    numpy \
    ipywidgets \
    dask[complete]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.0/80.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take

In [2]:
# setup and imports
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import cftime
import zarr
import fsspec
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import s3fs
import numpy as np
import ipywidgets as widgets
import dask

In [ ]:
meta = pd.read_csv("metadata.csv")
display(meta.head(10))

FileNotFoundError: [Errno 2] No such file or directory: 'metadata.csv'

In [ ]:
choice = int(input("\nEnter the number of the dataset you want to explore: "))
row = meta.iloc[choice]

year = int(input("Enter year (e.g., 2025): ").strip())
month = int(input("Enter month (01–12): "))
day = int(input("Enter day (01–31): "))

#print(row)

# Handle different file types and locations
if row["engine"] == "zarr":
    if row["inputFile"].startswith("s3://"):
        ds = xr.open_zarr("reference://", storage_options={"fo": row["kerchunkPath"], "remote_protocol": "s3", "remote_options": {"anon": True}, "asynchronous": False}, consolidated=False)
    else:
        ds = xr.open_zarr(row["inputFile"], consolidated=True)
elif row["engine"] == "h5netcdf":
    url = row["inputFile"].format(year=year, month=month, day=day) if "{" in row["inputFile"] else row["inputFile"]
    fs = fsspec.filesystem("s3", anon=True)
    if "*" in url or "?" in url:
        print(f"Searching for files matching pattern: {url}")
        # Use glob to find matching files
        matching_files = fs.glob(url)
        if not matching_files:
            raise FileNotFoundError(f"No files found matching pattern: {url}")
        # Use the first matching file
        url = f"s3://{matching_files[0]}"
        print(f"Found {len(matching_files)} file(s), using: {url}")
    s3_file = fs.open(url, mode="rb")
    ds = xr.open_dataset(s3_file, engine="h5netcdf")
else:
    ds = xr.open_dataset(row["inputFile"], engine=row["engine"])

#print(ds)

# Normalize coordinate names
coord_map = {c.lower(): c for c in ds.coords}
lat_name = coord_map.get("lat") or coord_map.get("latitude") or coord_map.get("lattitude")
lon_name = coord_map.get("lon") or coord_map.get("longitude") or coord_map.get("long")
time_name = coord_map.get("time")

# detect level dim from variable dims
extra = [d for d in ds[row["keyVariable"]].dims if d not in (time_name, lat_name, lon_name)]
level_name = extra[0] if extra else None

# level selection from metadata
chosen_level = None
if level_name:
    lv = np.array([float(x) for x in str(row.get("levelValues","")).replace(";",",").split(",") if x.strip()], float)
    default_level = float(np.nanmedian(lv))
    print(f"Available levels: {lv} {row.get('levelUnits','')}")
    raw = input(f"Enter {level_name} value (Enter for {default_level} {row.get('levelUnits','')}): ").strip()
    chosen_level = float(raw) if raw else default_level
    print(f"Using {level_name} = {chosen_level} {row.get('levelUnits','')}")

# time slice and data extraction
# Create target date from user input
target_date = pd.Timestamp(f"{year}-{month:02d}-{day:02d}")
# Find closest time in dataset
time_sel = ds[time_name].sel({time_name: target_date}, method="nearest")
map_sel = {time_name: time_sel}
if level_name:
    map_sel[level_name] = chosen_level

data = ds[row["keyVariable"]].sel(map_sel, method="nearest")

# normalize longitudes to [-180, 180]
if float(data[lon_name].max()) > 180:
    lon_norm = ((data[lon_name] + 180) % 360) - 180
    data = data.assign_coords({lon_name: lon_norm}).sortby(lon_name)

# light spatial smoothing
data_smooth = data.rolling({lat_name: 3, lon_name: 3}, center=True).mean()


# CREATE THE MAP
fig = plt.figure(figsize=(14, 7))
ax = plt.axes(projection=ccrs.PlateCarree())

# Plot the data
im = data_smooth.plot.contourf(ax=ax, levels=60, add_colorbar=False, transform=ccrs.PlateCarree())

# Add coastlines
ax.coastlines(linewidth=0.5, color='black')

# Set global extent to show full world
ax.set_global()

# Add gridlines with numerical labels only (no N/S/E/W)
import matplotlib.ticker as mticker

# Create custom formatters that show only numbers
def lon_formatter(x, pos):
    return f'{x:.0f}'

def lat_formatter(x, pos):
    return f'{x:.0f}'

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 10}
gl.ylabel_style = {'size': 10}
gl.xformatter = mticker.FuncFormatter(lon_formatter)
gl.yformatter = mticker.FuncFormatter(lat_formatter)

# Add colorbar
lvl_txt = f", {level_name}={chosen_level} {row.get('levelUnits','')}" if level_name else ""
if pd.isna(row['units']):
    plt.colorbar(im, label=f"{row['layerParameter']}")
else:
    plt.colorbar(im, label=f"{row['layerParameter']} ({row['units']})")

# Set title and labels
plt.title(f"{row['datasetName']}\n{row['keyVariable']} at {str(time_sel.values)[:10]}{lvl_txt}")
plt.xlabel("Longitude (-180°W to 180°E)")
plt.ylabel("Latitude (-90°S to 90°N)")
plt.tight_layout()
plt.show()

In [ ]:
# Simple coordinate input method (more reliable than clicking)
print("Enter coordinates for the zoomed view:")
lat_input = float(input("Enter latitude (e.g., 32.7): "))
lon_input = float(input("Enter longitude (e.g., -117.2): "))

# Create zoomed view
fig = plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
im = data_smooth.plot.contourf(ax=ax, levels=60, add_colorbar=False, transform=ccrs.PlateCarree())

ax.coastlines(linewidth=0.5, color='black')
ax.plot(lon_input, lat_input, 'ro', markersize=10, markeredgecolor='white', markeredgewidth=2, transform=ccrs.PlateCarree())

# Zoom into the area around the selected point
zoom_degrees = 10  # Adjust this value to control zoom level
ax.set_extent([lon_input - zoom_degrees, lon_input + zoom_degrees,
               lat_input - zoom_degrees, lat_input + zoom_degrees],
              crs=ccrs.PlateCarree())

# Add gridlines with numerical labels only (no N/S/E/W)
import matplotlib.ticker as mticker

# Create custom formatters that show only numbers
def lon_formatter(x, pos):
    return f'{x:.0f}'

def lat_formatter(x, pos):
    return f'{x:.0f}'

gl = ax.gridlines(draw_labels=True, alpha=0.5)
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 10}
gl.ylabel_style = {'size': 10}
gl.xformatter = mticker.FuncFormatter(lon_formatter)
gl.yformatter = mticker.FuncFormatter(lat_formatter)

# colorbar
if pd.isna(row['units']):
    plt.colorbar(im, label=f"{row['layerParameter']}")
else:
    plt.colorbar(im, label=f"{row['layerParameter']} ({row['units']})")

plt.title(f"{row['datasetName']}\n{row['keyVariable']} at {str(time_sel.values)[:10]}{lvl_txt}\nSelected: {lat_input:.2f}°N, {lon_input:.2f}°E")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.tight_layout()
plt.show()

# Extract value at selected point
point_data = data.sel({lat_name: lat_input, lon_name: lon_input}, method="nearest")
print(f"\nData value at selected point: {float(point_data.values):.3f} {row['units']}")


In [ ]:
# Time series analysis
from datetime import datetime
import s3fs

print("Creating time series for the selected coordinates...")

def create_time_series_plot(ts):
    """Create time series plot"""
    plt.figure(figsize=(12, 6))

    # If ts has multiple dimensions, select the chosen level
    if len(ts.dims) > 1 and level_name:
        ts_1d = ts.sel({level_name: chosen_level}, method="nearest")
    else:
        ts_1d = ts

    # Now plot the 1D time series
    ts_1d.plot(marker='o', linewidth=1.5, markersize=4)

    # Format title
    month_name = datetime.strptime(f"{month:02d}", '%m').strftime('%B')

    # Add level information to title if available
    if level_name and chosen_level is not None:
        level_text = f" at {level_name}={chosen_level} {row.get('levelUnits', '')}"
    else:
        level_text = ""

    plt.title(f"Time Series - {row['layerParameter']}{level_text} at ({lat_input:.2f}, {lon_input:.2f})\n{month_name} {year}", fontsize=14)

    # Set labels
    if pd.isna(row['units']):
        plt.ylabel(f"{row['layerParameter']}")
    else:
        plt.ylabel(f"{row['layerParameter']} ({row['units']})")
    plt.xlabel("Date")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Print summary statistics
    print(f"\nTime Series Summary:")
    print(f"Mean value: {float(ts_1d.mean()):.3f} {row['units']}")
    print(f"Max value: {float(ts_1d.max()):.3f} {row['units']}")
    print(f"Min value: {float(ts_1d.min()):.3f} {row['units']}")
    print(f"Data points: {len(ts_1d)}")

# Case 1: h5netcdf engine (S3 direct access)
if row["engine"] == "h5netcdf":
    try:
        print("Creating time series from S3 h5netcdf files...")
        fs = s3fs.S3FileSystem(anon=True)

        # Build file pattern based on the inputFile template
        base_pattern = row["inputFile"]

        # Handle different date format patterns
        if "{year:04d}" in base_pattern and "{month:02d}" in base_pattern and "{day:02d}" in base_pattern:
            # For daily data, replace day with wildcard but keep year/month as integers
            file_pattern = base_pattern.replace("{day:02d}", "*")
            file_pattern = file_pattern.format(year=year, month=month)
        elif "{year:04d}" in base_pattern and "{month:02d}" in base_pattern:
            # For monthly data, get the specific month
            file_pattern = base_pattern.format(year=year, month=month)
        else:
            print("Cannot determine file pattern for time series")
            file_pattern = None

        if file_pattern:
            # Remove s3:// prefix for glob
            if file_pattern.startswith("s3://"):
                file_pattern = file_pattern[5:]

            print(f"Searching for files matching: {file_pattern}")
            file_urls = fs.glob(file_pattern)

            if not file_urls:
                print(f"No files found for {year:04d}-{month:02d}")
            else:
                print(f"Found {len(file_urls)} files")

                # Open all files as a single dataset
                ds_ts = xr.open_mfdataset(
                    [fs.open(f, mode='rb') for f in file_urls],
                    engine="h5netcdf",
                    combine="by_coords",
                    parallel=True,
                    chunks={"time": 1}
                )

                # Extract data at the selected point using the correct coordinate names
                var_name = row["keyVariable"]
                point_data = ds_ts[var_name].sel({lat_name: lat_input, lon_name: lon_input}, method="nearest")
                ts = point_data.load()

                create_time_series_plot(ts)

    except Exception as e:
        print(f"Error with S3 h5netcdf files: {e}")

# Case 2: zarr engine (local or S3 with kerchunk)
elif row["engine"] == "zarr":
    try:
        print("Creating time series from zarr dataset...")
        var_name = row["keyVariable"]

        # Get all time steps using the correct coordinate names
        point_data = ds[var_name].sel({lat_name: lat_input, lon_name: lon_input}, method="nearest")
        ts = point_data.load()

        if len(ts) > 1:
            print(f"Found {len(ts)} time steps in zarr dataset")
            create_time_series_plot(ts)
        else:
            print("Only one time step available in zarr dataset")
    except Exception as e:
        print(f"Error with zarr dataset: {e}")

else:
    print(f"Time series not supported for engine: {row['engine']}")

In [ ]:
# Time series comparison using existing dataset
from datetime import datetime

print("Creating time series comparison...")

# Get comparison date from user
year2 = int(input("Enter comparison year (e.g., 2024): ").strip())
month2 = int(input("Enter comparison month (01–12): "))

def create_comparison_plot(ts1, ts2, label1, label2):
    """Create comparison time series plot"""
    plt.figure(figsize=(12, 6))

    # Extract day numbers for x-axis alignment
    if hasattr(ts1.time, 'dt'):
        days1 = ts1.time.dt.day
    else:
        days1 = ts1.time.values.astype('datetime64[D]').astype(int) % 100

    if hasattr(ts2.time, 'dt'):
        days2 = ts2.time.dt.day
    else:
        days2 = ts2.time.values.astype('datetime64[D]').astype(int) % 100

    # Plot both time series using day numbers as x-axis
    plt.plot(days1, ts1.values, marker='o', linewidth=1.5, markersize=4, label=label1)
    plt.plot(days2, ts2.values, marker='s', linewidth=1.5, markersize=4, label=label2)

    # Format title
    month_name1 = datetime.strptime(f"{month:02d}", '%m').strftime('%B')
    month_name2 = datetime.strptime(f"{month2:02d}", '%m').strftime('%B')

    # Add level information to title if available
    if level_name and chosen_level is not None:
        level_text = f" at {level_name}={chosen_level} {row.get('levelUnits', '')}"
    else:
        level_text = ""

    plt.title(f"Time Series Comparison - {row['layerParameter']}{level_text} at ({lat_input:.2f}, {lon_input:.2f})\n{month_name1} {year} vs {month_name2} {year2}", fontsize=14)

    # Set labels
    if pd.isna(row['units']):
        plt.ylabel(f"{row['layerParameter']}")
    else:
        plt.ylabel(f"{row['layerParameter']} ({row['units']})")
    plt.xlabel("Day of Month")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Print comparison statistics
    print(f"\nComparison Summary:")
    print(f"{month_name1} {year}: Mean={float(ts1.mean()):.3f}, Max={float(ts1.max()):.3f}, Min={float(ts1.min()):.3f}")
    print(f"{month_name2} {year2}: Mean={float(ts2.mean()):.3f}, Max={float(ts2.max()):.3f}, Min={float(ts2.min()):.3f}")
    print(f"Difference (Mean): {float(ts2.mean() - ts1.mean()):.3f} {row['units']}")

# Case 1: h5netcdf engine (S3 direct access)
if row["engine"] == "h5netcdf":
    try:
        print("Creating time series from S3 h5netcdf files...")
        fs = s3fs.S3FileSystem(anon=True)

        # Build file pattern based on the inputFile template
        base_pattern = row["inputFile"]

        # Handle different date format patterns
        if "{year:04d}" in base_pattern and "{month:02d}" in base_pattern and "{day:02d}" in base_pattern:
            # For daily data, replace day with wildcard but keep year/month as integers
            file_pattern = base_pattern.replace("{day:02d}", "*")
            file_pattern = file_pattern.format(year=year2, month=month2)
        elif "{year:04d}" in base_pattern and "{month:02d}" in base_pattern:
            # For monthly data, get the specific month
            file_pattern = base_pattern.format(year=year2, month=month2)
        else:
            print("Cannot determine file pattern for time series")
            file_pattern = None

        if file_pattern:
            # Remove s3:// prefix for glob
            if file_pattern.startswith("s3://"):
                file_pattern = file_pattern[5:]

            print(f"Searching for files matching: {file_pattern}")
            file_urls = fs.glob(file_pattern)

            if not file_urls:
                print(f"No files found for {year2:04d}-{month2:02d}")
            else:
                print(f"Found {len(file_urls)} files")

                # Open all files as a single dataset
                ds_ts2 = xr.open_mfdataset(
                    [fs.open(f, mode='rb') for f in file_urls],
                    engine="h5netcdf",
                    combine="by_coords",
                    parallel=True,
                    chunks={"time": 1}
                )

                # Extract data at the selected point using the correct coordinate names
                var_name = row["keyVariable"]
                point_data = ds_ts2[var_name].sel({lat_name: lat_input, lon_name: lon_input}, method="nearest")
                ts2 = point_data.load()

                create_comparison_plot(ts, ts2, "Current", "Comparison")

    except Exception as e:
        print(f"Error with S3 h5netcdf files: {e}")